In [1]:
import requests
import spacy
import re
import spacy_transformers
import torch
from spacytextblob.spacytextblob import SpacyTextBlob
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer

In [2]:
nlp = spacy.load('de_dep_news_trf')
nlp.add_pipe('spacytextblob')

In [3]:
response = requests.get("https://search.dip.bundestag.de/api/v1/drucksache-text?f.dokumentnummer=20/3450&apikey=GmEPb1B.bfqJLIhcGAsH9fTJevTglhFpCoZyAAAdhp")
print(response)

<Response [200]>


In [3]:
response.json()

{'numFound': 1,
 'documents': [{'id': '263032',
   'drucksachetyp': 'Kleine Anfrage',
   'dokumentart': 'Drucksache',
   'autoren_anzahl': 0,
   'typ': 'Dokument',
   'vorgangsbezug_anzahl': 1,
   'dokumentnummer': '20/3450',
   'wahlperiode': 20,
   'herausgeber': 'BT',
   'datum': '2022-09-20',
   'titel': 'Ergebnisse des zweiten Stresstests',
   'text': 'Kleine Anfrage\nder Fraktion der CDU/CSU\nErgebnisse des zweiten Stresstests\nDie vier deutschen Übertragungsnetzbetreiber 50Hertz, Amprion, TenneT und \nTransnet haben am 5. September 2022 im Rahmen einer Pressekonferenz die \nErgebnisse des zweiten Netzstresstests (zweite Sonderanalyse Winter \n2022/2023) vorgestellt. Der Bundesminister für Wirtschaft und Klimaschutz \nDr. Robert Habeck hat im Anschluss an die Vorstellung der Ergebnisse seine \nSchlüsse aus dem Stresstest dargelegt (https://www.bmwk.de/Redaktion/DE/Pr\nessemitteilungen/2022/09/20220905-stresstest-zum-stromsystem.html). Danach \nsollen „die beiden AKW Isar 2 und Ne

In [4]:
data = response.json()

In [21]:
print("Type:", type(response.json()))
print("Type:", type(data))
print("Type:", type(data['documents']))
print("Type:", type(data['documents'][0]))

Type: <class 'dict'>
Type: <class 'dict'>
Type: <class 'list'>
Type: <class 'dict'>


In [22]:
print(data['documents'][0]['text'])

Kleine Anfrage
der Fraktion der CDU/CSU
Ergebnisse des zweiten Stresstests
Die vier deutschen Übertragungsnetzbetreiber 50Hertz, Amprion, TenneT und 
Transnet haben am 5. September 2022 im Rahmen einer Pressekonferenz die 
Ergebnisse des zweiten Netzstresstests (zweite Sonderanalyse Winter 
2022/2023) vorgestellt. Der Bundesminister für Wirtschaft und Klimaschutz 
Dr. Robert Habeck hat im Anschluss an die Vorstellung der Ergebnisse seine 
Schlüsse aus dem Stresstest dargelegt (https://www.bmwk.de/Redaktion/DE/Pr
essemitteilungen/2022/09/20220905-stresstest-zum-stromsystem.html). Danach 
sollen „die beiden AKW Isar 2 und Neckarwestheim bis Mitte April 2023 noch 
zur Verfügung stehen, um, falls nötig, über den Winter einen zusätzlichen Bei-
trag im Stromnetz in Süddeutschland 2022/23 leisten zu können.“ Eine einheit-
liche Auffassung der Bundesregierung zu diesen Erläuterungen des Bundes-
wirtschaftsministers scheint es bisher nicht zu geben (https://www.tagesscha
u.de/inland/habeck-stre

In [5]:
doc = nlp(data['documents'][0]['text'])

In [6]:
doc._.polarity

0.09999999999999999

In [7]:
for span in doc.sents:
    print(span.text, span._.polarity)
    print('')

Kleine Anfrage
der Fraktion der CDU/CSU
 0.0

Ergebnisse des zweiten Stresstests 0.0


 0.0

Die vier deutschen Übertragungsnetzbetreiber 50Hertz, Amprion, TenneT und 
Transnet haben am 5. September 2022 im Rahmen einer Pressekonferenz die 
Ergebnisse des zweiten Netzstresstests (zweite Sonderanalyse Winter 
2022/2023) vorgestellt. 0.0

Der Bundesminister für Wirtschaft und Klimaschutz 
Dr. Robert Habeck hat im Anschluss an die Vorstellung der Ergebnisse seine 
Schlüsse aus dem Stresstest dargelegt (https://www.bmwk.de/Redaktion/DE/Pr
essemitteilungen/2022/09/20220905-stresstest-zum-stromsystem.html). 0.0

Danach 
sollen „die beiden AKW Isar 2 und Neckarwestheim bis Mitte April 2023 noch 
zur Verfügung stehen, um, falls nötig, über den Winter einen zusätzlichen Bei-
trag im Stromnetz in Süddeutschland 2022/23 leisten zu können. 0.0

“ Eine einheit-
liche Auffassung der Bundesregierung zu diesen Erläuterungen des Bundes-
wirtschaftsministers scheint es bisher nicht zu geben (https://www

In [8]:
for span in doc.sents:
    blob = TextBlob(span.text, analyzer=NaiveBayesAnalyzer())
    print(span, blob.sentiment)
    print('')

Kleine Anfrage
der Fraktion der CDU/CSU
 Sentiment(classification='neg', p_pos=0.38888888888888884, p_neg=0.6111111111111108)

Ergebnisse des zweiten Stresstests Sentiment(classification='pos', p_pos=0.75, p_neg=0.2499999999999997)


 Sentiment(classification='pos', p_pos=0.5, p_neg=0.5)

Die vier deutschen Übertragungsnetzbetreiber 50Hertz, Amprion, TenneT und 
Transnet haben am 5. September 2022 im Rahmen einer Pressekonferenz die 
Ergebnisse des zweiten Netzstresstests (zweite Sonderanalyse Winter 
2022/2023) vorgestellt. Sentiment(classification='pos', p_pos=0.9605433376455376, p_neg=0.039456662354463025)

Der Bundesminister für Wirtschaft und Klimaschutz 
Dr. Robert Habeck hat im Anschluss an die Vorstellung der Ergebnisse seine 
Schlüsse aus dem Stresstest dargelegt (https://www.bmwk.de/Redaktion/DE/Pr
essemitteilungen/2022/09/20220905-stresstest-zum-stromsystem.html). Sentiment(classification='neg', p_pos=0.333795367688436, p_neg=0.6662046323115643)

Danach 
sollen „die beiden A


Wenn ja, inwiefern?
 Sentiment(classification='pos', p_pos=0.5, p_neg=0.5)

Wenn nein, warum nicht? Sentiment(classification='pos', p_pos=0.5, p_neg=0.5)


 Sentiment(classification='pos', p_pos=0.5, p_neg=0.5)

12. Sentiment(classification='pos', p_pos=0.5, p_neg=0.5)

Hat die Bundesregierung eine Position zu der Aussage der Wirtschaftswei-
sen Sentiment(classification='neg', p_pos=0.1453256623528475, p_neg=0.8546743376471518)

Veronica Sentiment(classification='neg', p_pos=0.4642857142857146, p_neg=0.5357142857142856)

Grimm Sentiment(classification='neg', p_pos=0.2499999999999997, p_neg=0.75)

, Sentiment(classification='pos', p_pos=0.5, p_neg=0.5)

die Kernkraftwerke sollten laufen und nicht nur in 
Bereitschaft sein, wie es aktuell geplant ist, denn „nur dann gibt es einen 
senkenden Effekt auf den Strompreis“, erarbeitet, und wenn ja, welche, 
und welche Schlussfolgerungen zieht sie daraus für ihr eigenes Handeln 
(https://www.wiwo.de/politik/deutschland/atom-reserve-wirtschafts

Wie soll nach Einschätzung der Bundesregierung eine mögliche zeitliche 
Lücke geschlossen werden, um die Netzstabilität zu sichern? Sentiment(classification='neg', p_pos=0.3645833333333331, p_neg=0.6354166666666669)


 Sentiment(classification='pos', p_pos=0.5, p_neg=0.5)

27. Sentiment(classification='pos', p_pos=0.5, p_neg=0.5)

Welche „notwendigen technischen Anforderungen der Atomkraft“ (https://
www.bmwk.de/Redaktion/DE/Downloads/F/faq-zweiter-stresstest-massna
hmen-sicherung-stromnetz-stabilitat.pdf?__blob=publicationFile&v=6, 
S. 6) müssen bei der Ausgestaltung der „Einsatzreserve“ berücksichtigt 
werden? Sentiment(classification='neg', p_pos=0.38888888888888884, p_neg=0.6111111111111108)


 Sentiment(classification='pos', p_pos=0.5, p_neg=0.5)

28. Sentiment(classification='pos', p_pos=0.5, p_neg=0.5)

Sind die anstelle des Kernkraftwerks Emsland vorgesehenen zusätzlichen 
Ölkraftwerke in Form von Kraftwerksschiffen, sogenannte Power-Barges 
(https://www.bmwk.de/Redaktion/DE/Do


 Sentiment(classification='pos', p_pos=0.5, p_neg=0.5)

48. Sentiment(classification='pos', p_pos=0.5, p_neg=0.5)

Wie viele Kohlekraftwerke (in Megawatt) müssen nach Kenntnis der 
Bundesregierung anstelle der drei laufenden Kernkraftwerke ab Anfang 
2023 in den Markt zurückkehren?
49. Sentiment(classification='pos', p_pos=0.6562499999999998, p_neg=0.3437499999999998)

Wieso wurde das erste Ergebnis der „Sonderanalyse Winter 2022/2023“ 
der Übertragungsnetzbetreiber, wie von dem Staatssekretär im BMWK 
Dr. Patrick Graichen in der Sondersitzung des Ausschusses für Klima-
schutz und Energie am 31. August 2022 bestätigt, wieder zur Überarbei-
tung an die Übertragungsnetzbetreiber zurückgegeben, und wer traf diese 
Entscheidung?
50. Sentiment(classification='pos', p_pos=0.9965995958360757, p_neg=0.003400404163926184)

Hat die Bundesregierung vor ihrer Ankündigung des Vorhaltens von Kern-
kraftwerken in der „Einsatzreserve“ evaluiert, ob dieses Vorhalten recht-
lich mit der in § 7 Absatz 1